In [ ]:
import numpy as np
import random
import xgi
from itertools import combinations
from tqdm.auto import tqdm
import networkx as nx
import pickle

from ho_info_metrics.metrics import o_information_lagged_all

## ISING MODEL

In [ ]:
from ising import *

In [ ]:
N = 200
ps = [0.01, 0.0005]

S = xgi.random_simplicial_complex(N, ps, seed=1)

# 2-simplices
two_simplices = S.edges.filterby("size", 3).members()
two_simplices = [sorted(list(i)) for i in two_simplices]
print(len(two_simplices))
# 3-cliques
skeleton = xgi.convert_to_graph(S)
cliques = nx.find_cliques(skeleton)
cliques = [sorted(list(i)) for i in cliques]
three_cliques = [i for i in cliques if (len(i) == 3 and i not in two_simplices)]
print(len(three_cliques))

# random triplets
all_triplets = [
    sorted(list(i))
    for i in combinations(skeleton.nodes, 3)
    if (i not in two_simplices and i not in three_cliques)
]
num_to_select = len(two_simplices)  # set the number to select here.
random_triplets = random.sample(all_triplets, num_to_select)
print(len(random_triplets))

In [ ]:
edges = [list(i) for i in S.edges.filterby("order", 1).members()]
triangles = [list(i) for i in S.edges.filterby("order", 2).members()]

In [ ]:
T = 6
J1 = 0.2
J2s = [0.2, 3.5, 8]
for i, J2 in enumerate(J2s):
    print(">>> Simulating")
    ts, _, _ = ising_dynamic(N, edges, triangles, J1, J2, 100000, T=6)
    ts = ts[:, 40000:]
    print(">>> Computing dO-information")
    two_simplices_lin = []

    bar_length = len(two_simplices)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("2-simplices")
        for i in two_simplices:
            X = np.vstack((ts[i[0]], ts[i[1]], ts[i[2]]))
            O_infos_3 = o_information_lagged_all(X, estimator="cat_ent")
            two_simplices_lin.append(O_infos_3)
            pbar.update(1)
    file_name = (
        "./ising_results/ising_J1_"
        + str(J1)
        + "_J2_"
        + str(J2)
        + "_T_"
        + str(T)
        + "_N_"
        + str(N)
        + "_dOinfo_two_simplices.npy"
    )
    np.save(file_name, two_simplices_lin)

    three_cliques_lin = []

    bar_length = len(three_cliques)
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("3-cliques")
        for i in three_cliques:
            X = np.vstack((ts[i[0]], ts[i[1]], ts[i[2]]))
            O_infos_3 = o_information_lagged_all(X, estimator="cat_ent")
            three_cliques_lin.append(O_infos_3)
            pbar.update(1)
    file_name = (
        "./ising_results/ising_J1_"
        + str(J1)
        + "_J2_"
        + str(J2)
        + "_T_"
        + str(T)
        + "_N_"
        + str(N)
        + "_dOinfo_three_cliques.npy"
    )
    np.save(file_name, three_cliques_lin)

    bar_length = len(random_triplets)
    random_triplets_lin = []
    with tqdm(total=bar_length) as pbar:
        pbar.set_description("Random triplets")
        for i in random_triplets:
            X = np.vstack((ts[i[0]], ts[i[1]], ts[i[2]]))
            O_infos_3 = o_information_lagged_all(X, estimator="cat_ent")
            random_triplets_lin.append(O_infos_3)
            pbar.update(1)
    file_name = (
        "./ising_results/ising_J1_"
        + str(J1)
        + "_J2_"
        + str(J2)
        + "_T_"
        + str(T)
        + "_N_"
        + str(N)
        + "_dOinfo_random_triplets.npy"
    )
    np.save(file_name, random_triplets_lin)

# SIMPLAGION MODEL

In [ ]:
from higher_order_contagion import *

In [ ]:
# Simplicial Complex parameters
N = 300  # number of nodes
k1_init = 20  # average degree from wich we construct the network
k2_init = 7  # average hyper-degree (mean number of triangles per node)
p1, p2 = get_p1_and_p2_correction(k1_init, k2_init, N)

G, node_neighbors_dict, triangles_list = generate_my_simplicial_complex_d2(N, p1, p2)

# Real average degree and hyper-degree of the simplicial complex
k1 = (
    1.0 * sum([len(v) for v in node_neighbors_dict.values()]) / len(node_neighbors_dict)
)
k2 = 3.0 * len(triangles_list) / len(node_neighbors_dict)

print(k1, k2)

I0_percentage = 30.0  # percentage of initial infected nodes

mySimplagionModel = SimplagionModel(node_neighbors_dict, triangles_list, I0_percentage)

initial_infected = mySimplagionModel.initial_setup(print_status=True)
# create and save the initial infected nodes list

t_max = 10000  # number of time steps
mu = 0.8  # recovery rate
lambdas = [
    (2.5, 0.0),
    (2.5, 2.5),
    (2.5, 7.0),
]  # pairs of rescaled (simple, simplicial) infectivity
simplagion_results = {}  # dictionary containing the results

for (lambda1, lambda2) in lambdas:
    beta1 = lambda1 * mu / k1  # simple-infection rate
    beta2 = lambda2 * mu / k2  # simplicial-infection rate
    mySimplagionModel.initial_setup(fixed_nodes_to_infect=initial_infected)
    one_result = mySimplagionModel.run(t_max, beta1, beta2, mu, print_status=True)

    args = (lambda1, lambda2)
    simplagion_results[args] = one_result

# NOTE: if all the nodes get infected we continue the recovery process.
# NOTE2: if all the nodes are 'S' at a certain timestep we restart the dynamical process with a new random seed.

print([simplagion_results[lambdas[i]].shape for i in range(3)])

In [ ]:
%%time

frequency_O_info_simplices = {}

for l in lambdas:
    data_O_infos_3 = {}
    for triplet in triangles_list:
        X_new = simplagion_results[l][triplet, :]

        data_O_infos_3[triplet] = o_information_lagged_all(X_new, estimator="cat_ent")

    frequency_O_info_simplices[l] = data_O_infos_3
# simplagion_O_info_simplices is a dictionary whose keys are the lambdas configurations and the values are
# dictionaries containing the 2-simplices as keys and the corresponding dO-infos as values.

pickle.dump(
    frequency_O_info_simplices,
    open("./simplagion_results/frequency_O_info_simplices.pickle", "wb"),
)

In [ ]:
%%time

frequency_O_info_cliques = {}

cliques_3_to_convert = [clq for clq in nx.enumerate_all_cliques(G) if len(clq)==3]
cliques_3 = list([tuple(t) for t in cliques_3_to_convert])

for l in lambdas:
    data_O_infos_3 = {}
    counter = 1
    for tri in cliques_3:
        if counter > len(triangles_list) : break
        if tri in triangles_list: continue
        else:
            X_new = simplagion_results[l][tri, :]
            data_O_infos_3[tri]= o_information_lagged_all(X_new, estimator='cat_ent')  
            counter +=1
    
    frequency_O_info_cliques[l] =  data_O_infos_3

pickle.dump(frequency_O_info_cliques, open("./simplagion_results/frequency_O_info_cliques.pickle', 'wb'))

In [ ]:
%%time

frequency_O_info_triplets = {}

random_triplets = list(combinations(G.nodes(), 3))
random.shuffle(random_triplets)

for l in lambdas:
    data_O_infos_3 = {}
    counter = 1
    for tri in random_triplets:
        if counter > len(triangles_list):
            break
        if tri in triangles_list or tri in cliques_3:
            continue
        else:
            X_new = simplagion_results[l][tri, :]
            data_O_infos_3[tri] = o_information_lagged_all(X_new, estimator="cat_ent")
            counter += 1

    frequency_O_info_triplets[l] = data_O_infos_3

pickle.dump(
    frequency_O_info_triplets,
    open("./simplagion_results/frequency_O_info_triplets.pickle", "wb"),
)